TO DO

- add an optional constraint to allow only one defender from any given team (risk mitigation)

In [1]:
import pandas as pd
import numpy as np
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

In [2]:
# function to solve the optimization problem
def optimize(df, max_price, expected_column):
    
    # PRELIMINARIES
    
    # Create the model
    model = LpProblem(name="FPL", sense=LpMaximize)    
    variables = [LpVariable(name=f'{ix}', cat='Binary') for ix in df.index]
    prices = [df.loc[ix,'now_cost']/10.0 for ix in df.index]
    # measure of player quality
    expected_points = [df.loc[ix,expected_column] for ix in df.index]
    goalkeepers = [1.0 if df.loc[ix,'element_type']==1 else 0.0 for ix in df.index]
    defenders = [1.0 if df.loc[ix,'element_type']==2 else 0.0 for ix in df.index]
    midfielders = [1.0 if df.loc[ix,'element_type']==3 else 0.0 for ix in df.index]
    forwards = [1.0 if df.loc[ix,'element_type']==4 else 0.0 for ix in df.index]

    # CONSTRAINTS

    # select 11 players
    model += lpSum(variables) == 11
    # set maximum price for starting 11
    model += np.dot(prices,variables) <= max_price
    # only 1 goalkeeper
    model += np.dot(goalkeepers,variables) == 1
    # at least 3 defenders
    model += np.dot(defenders,variables) >= 3
    # at most 5 defenders
    model += np.dot(defenders,variables) <= 5
    # at most 5 midfielders
    model += np.dot(midfielders,variables) <= 5
    # at least 1 forward
    model += np.dot(forwards,variables) >= 1
    # at most 3 forwards
    model += np.dot(forwards,variables) <= 3
    
    # OBJECTIVE
    # if possible, SHOULD ADD CAPTAIN'S DOUBLE POINTS TO THE OBJECTIVE
    model += np.dot(expected_points,variables)

    # SOLVE OPTIMIZATION
    
    status = model.solve()
    print(f'Status: {LpStatus[model.status]}')
    #print(f'Mean total points per gameweek: {model.objective.value()}')

    players = [int(str(var)) for var in model.variables() if var.value()==1]
    dream_team = df.loc[players]
    cost = (dream_team['now_cost']/10.0).sum()
    exp_points = dream_team[expected_column].sum() + dream_team[expected_column].max()
    print(f'Cost: {cost}')
    if expected_column == 'adjusted points per game':
        print(f'Expected points per week: {exp_points}')
    elif expected_column == 'xPoints_next10':
        exp_points = exp_points / 10
        print(f'Expected points per week: {exp_points}')
    else:
        print(f'Expected points: {exp_points}')
    display(dream_team[['element_type','web_name','now_cost','games played',expected_column]].sort_values('element_type'))

In [3]:
gameweek = 8
# fetch FPL data
filepath = '../data/fpl/data_week' + str(gameweek) + '.csv'
df = pd.read_csv(filepath, index_col=0)
df

,adjusted points,adjusted points per game,assists,assists_week1,assists_week2,assists_week3,assists_week4,assists_week5,assists_week6,assists_week7,...,xPoints week 2,xPoints week 3,xPoints week 4,xPoints week 5,xPoints week 6,xPoints week 7,xPoints week 8,xPoints_next10,yellow_cards,gameweek 18 prediction
id,,,,,,,,,,,,,,,,,,,,,
1,6.304599,2.101533,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,2.220093,1.089483,NaN,NaN,NaN,NaN,NaN,38.105189,0,4.017078
2,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.830863,0,3.405957
3,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.168214,0,2.478769
4,29.100000,4.157143,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,1.000000,10.5,3.6,3.8,2.4,6.8,38.682989,0,4.271172
5,3.604599,1.201533,0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,...,1.520093,-0.910517,1.0,NaN,NaN,NaN,NaN,19.925263,1,2.200027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.791775,0,2.354507
548,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.791775,0,2.354507
549,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.845476,0,2.514374


In [4]:
# only consider players that have played at least some minimum number of games
df = df[df['games played']>=3]
df

,adjusted points,adjusted points per game,assists,assists_week1,assists_week2,assists_week3,assists_week4,assists_week5,assists_week6,assists_week7,...,xPoints week 2,xPoints week 3,xPoints week 4,xPoints week 5,xPoints week 6,xPoints week 7,xPoints week 8,xPoints_next10,yellow_cards,gameweek 18 prediction
id,,,,,,,,,,,,,,,,,,,,,
1,6.304599,2.101533,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,2.220093,1.089483,NaN,NaN,NaN,NaN,NaN,38.105189,0,4.017078
4,29.100000,4.157143,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,1.000000,10.500000,3.600000,3.800000,2.400000,6.800000,38.682989,0,4.271172
5,3.604599,1.201533,0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,...,1.520093,-0.910517,1.000000,NaN,NaN,NaN,NaN,19.925263,1,2.200027
6,5.600000,1.866667,0,NaN,NaN,0.0,NaN,NaN,NaN,0.0,...,NaN,1.000000,NaN,NaN,NaN,1.000000,3.600000,27.041798,0,3.402598
7,5.568200,1.392050,0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,...,2.055023,-2.000000,NaN,NaN,2.332871,NaN,NaN,18.799779,0,2.003818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439,25.258904,3.157363,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.892521,4.195247,4.274923,1.328340,3.204777,4.195247,2.931484,34.983427,1,3.409467
445,5.300000,1.060000,0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,...,1.000000,1.000000,NaN,1.000000,NaN,NaN,1.300000,21.028030,0,1.977102
461,24.073545,4.012257,0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,4.323130,5.548812,4.418731,2.382085,NaN,5.048812,NaN,37.041169,0,3.492197


In [5]:
optimize(df, 83.1, 'gameweek 9 prediction')

Status: Optimal
Cost: 82.4
Expected points: 66.42819499969482


,element_type,web_name,now_cost,games played,gameweek 9 prediction
id,,,,,
170,1,Pickford,50,7.0,4.621417
119,2,Azpilicueta,62,8.0,5.062356
122,2,Alonso,59,6.0,4.830716
237,2,Alexander-Arnold,75,6.0,5.282769
173,3,Doucouré,57,8.0,4.849048
233,3,Salah,128,8.0,6.032595
482,3,Townsend,57,8.0,5.178554
484,3,Gray,58,8.0,4.867161
177,4,Calvert-Lewin,80,3.0,5.974237


In [6]:
optimize(df, 83.1, 'adjusted points per game')

Status: Optimal
Cost: 83.1
Expected points per week: 76.53994230827007


,element_type,web_name,now_cost,games played,adjusted points per game
id,,,,,
475,1,Sá,50,8.0,4.403738
237,2,Alexander-Arnold,75,6.0,6.738858
256,2,Cancelo,63,8.0,5.996296
259,2,Laporte,55,6.0,4.957712
91,2,Pinnock,46,8.0,4.711622
230,3,Mané,119,8.0,6.437673
233,3,Salah,128,8.0,8.375673
268,3,Torres,68,4.0,6.228011
419,3,Benrahma,66,8.0,5.163121


In [7]:
df = df.drop([259,268])
optimize(df, 83.1, 'adjusted points per game')

Status: Optimal
Cost: 82.89999999999999
Expected points per week: 75.34959596075626


,element_type,web_name,now_cost,games played,adjusted points per game
id,,,,,
475,1,Sá,50,8.0,4.403738
237,2,Alexander-Arnold,75,6.0,6.738858
256,2,Cancelo,63,8.0,5.996296
91,2,Pinnock,46,8.0,4.711622
230,3,Mané,119,8.0,6.437673
233,3,Salah,128,8.0,8.375673
240,3,Jota,75,7.0,5.298441
482,3,Townsend,57,8.0,4.903121
96,3,Mbeumo,55,8.0,4.956937


In [8]:
optimize(df, 83.1, 'xPoints_next10')

Status: Optimal
Cost: 82.69999999999999
Expected points per week: 62.86724655628204


,element_type,web_name,now_cost,games played,xPoints_next10
id,,,,,
334,1,McCarthy,45,8.0,43.549747
237,2,Alexander-Arnold,75,6.0,51.970994
256,2,Cancelo,63,8.0,47.936861
76,2,Jansson,47,8.0,41.596666
91,2,Pinnock,46,8.0,43.713875
210,3,Tielemans,64,8.0,47.661609
233,3,Salah,128,8.0,60.255423
96,3,Mbeumo,55,8.0,44.782001
205,4,Vardy,106,8.0,60.451847
